Given function is:$$f(x)=\sqrt{x_1^2+4}+\sqrt{x_2^2+4}$$
gradient of $f(x)$ can be written as:
$$\nabla f(x)=\begin{bmatrix}
  \frac{x_1}{\sqrt{x_1^2 +4}} & \frac{x_2}{\sqrt{x_2^2 +4}}
\end{bmatrix}$$
Hessian matrix can be written as:
$$\nabla^2 f(x)=\begin{bmatrix}
  \frac{4}{(x_1^2 +4)^{3/2}} & 0\\
  0 & \frac{4}{(x_2^2 +4)^{3/2}}
\end{bmatrix}$$

In [ ]:
import numpy as np 

In [ ]:
#method to find Hessian matrix: Complete the code
def evalh(x): 
  assert type(x) is np.ndarray 
  assert len(x) == 2
  return np.array([[4/(x[0]**2 + 4)**1.5,0] , [0,4/(x[1]**2 + 4)**1.5]])

In [ ]:
#method to find the condition number of any square matrix: : Complete the code
def find_condition_number(A):
  assert type(A) is np.ndarray
  assert A.shape[0] == A.shape[1]
  k=max(np.linalg.eig(A)[0])/min(np.linalg.eig(A)[0])
  return k

In [ ]:
#Now we will define a Python function which will compute and return the objective function value 
def evalf(x):  
  #Input: x is a numpy array of size 2 
  assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments 
  #after checking if the argument is valid, we can compute the objective function value
  #compute the function value and return it 
  return (x[0]**2+4)**0.5 + (x[1]**2+4)**0.5 

In [ ]:
#Now we will define a Python function which will compute and return the gradient value as a numpy array 
def evalg(x):  
  #Input: x is a numpy array of size 2 
  assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments 
  #after checking if the argument is valid, we can compute the gradient value
  #compute the gradient value and return it 
  return np.array([x[0]/(x[0]**2+4)**0.5 ,x[1]/(x[1]**2+4)**0.5 ])

In [ ]:
#The method defines a way to construct D_k matrix used in scaling the gradient in the modified gradient descent scheme
def compute_D_k(x):
  assert type(x) is np.ndarray
  assert len(x) == 2
  #compute and return D_k
  d_k = np.array([[1/evalh(x)[0][0], 0],[0, 1/evalh(x)[1][1]]])
  return d_k

In [ ]:
#The method defines a way to construct D_k matrix used in scaling the gradient in the modified gradient descent scheme
def compute_D_k_nondiagonal(x):
  assert type(x) is np.ndarray
  assert len(x) == 2
  #compute and return D_k_1 for non diagonal D
  if np.linalg.det(evalh(x))==0:
    raise ValueError('Determinant is 0')
  d_k = np.linalg.inv(evalh(x))
  return d_k

In [ ]:
#Complete the module to compute the steplength by using the closed-form expression
def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 

  alpha = alpha_start
  #implement the backtracking line search
  p= -gradf
  while(evalf(x+alpha*p) > evalf(x) + gamma*alpha*np.dot(gradf.transpose(),p)):
    alpha = rho*alpha


  #print('final step length:',alpha)
  return alpha

In [ ]:
def compute_steplength_backtracking_scaled_direction(x, gradf, direction, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(direction) is np.ndarray and len(direction) == 2 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  alpha = alpha_start
  while evalf(x + alpha*direction) > ( evalf(x) +  (gamma*alpha*np.dot(gradf,direction)) ):
    alpha = rho*alpha
  return alpha

In [ ]:
#line search type 
EXACT_LINE_SEARCH = 1
BACKTRACKING_LINE_SEARCH = 2
CONSTANT_STEP_LENGTH = 3
BACKTRACKING_LINE_SEARCH_SCALING = 4

In [ ]:
#complete the code for gradient descent to find the minimizer
def find_minimizer_gd(start_x, tol, line_search_type,*args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  
  #Complete the code   
  A = np.array([[1500, 2],[2,1]])
  x = start_x
  g_x = evalg(x)

  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    # print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)
    condition_num_arr = []
  k = 0
  # print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    condition_num_arr.append(find_condition_number(evalh(x)))
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma) #call the new function you wrote to compute the steplength
      #raise ValueError('BACKTRACKING LINE SEARCH NOT YET IMPLEMENTED')
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here   
    x = np.subtract(x, np.multiply(step_length,g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point

    # print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x,k, condition_num_arr

In [ ]:
#complete the code for gradient descent with scaling to find the minimizer

def find_minimizer_gdscaling(start_x, tol, line_search_type,*args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  A = np.array([[1500, 2],[2,1]])
  x = start_x
  g_x = evalg(x)
  d_k = compute_D_k(x)
  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH_SCALING):
    alpha = args[0]
    rho = args[1]
    gamma = args[2]
    #print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)

  k = 0
  # print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    direction = -np.matmul(d_k,g_x)
    if line_search_type == BACKTRACKING_LINE_SEARCH_SCALING:
      step_length = compute_steplength_backtracking_scaled_direction(x,g_x,direction,alpha,rho,gamma)
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here   
    x = np.subtract(x, np.matmul(np.multiply(step_length,g_x),d_k)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point

    # print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x ,k

In [ ]:
#complete the code for gradient descent with scaling to find the minimizer
import scipy
from scipy.linalg import sqrtm
def find_minimizer_gdscaling_non_diagonal(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  x = start_x
  g_x = evalg(x)

  condition_number_scaling = []

  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH_SCALING):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]

  k = 0
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    d_k = compute_D_k_nondiagonal(x)
    direction = np.matmul(-d_k,g_x)
    
    p = scipy.linalg.sqrtm(d_k)
    hessn = np.matmul(np.matmul(p,evalh(x)),p)
    condition_number_scaling.append(find_condition_number(hessn))

    if line_search_type == BACKTRACKING_LINE_SEARCH_SCALING:
      step_length = compute_steplength_backtracking_scaled_direction(x, g_x,direction, alpha_start, rho, gamma) #call the new function you wrote to compute the steplength 
    elif line_search_type == CONSTANT_STEP_LENGTH:
      step_length = 1.0
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here  
    x = np.subtract(x, np.multiply(step_length,np.matmul(d_k, g_x))) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point
    # print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  return x, k, condition_number_scaling

## **Answer 2:**

In [ ]:
my_tol = 1e-9
alpha = 1.0
rho = 0.5
gamma = 0.5
my_start_x = np.array([2.0,2.0])

In [ ]:
#Compute gradient descent with scaling and with backtracking line search 

print("Newtons method with scaling and constant step length")
x_opt_bls, k, condition2 = find_minimizer_gdscaling_non_diagonal(my_start_x, my_tol, CONSTANT_STEP_LENGTH, 1.0, 0.5, 0.5)


Streaming output truncated to the last 5000 lines.
iter: 9487  x: [-2. -2.]  f(x): 5.656854249492381  grad at x: [-0.70710678 -0.70710678]  gradient norm: 0.9999999999999999
iter: 9488  x: [2. 2.]  f(x): 5.656854249492381  grad at x: [0.70710678 0.70710678]  gradient norm: 0.9999999999999999
iter: 9489  x: [-2. -2.]  f(x): 5.656854249492381  grad at x: [-0.70710678 -0.70710678]  gradient norm: 0.9999999999999999
iter: 9490  x: [2. 2.]  f(x): 5.656854249492381  grad at x: [0.70710678 0.70710678]  gradient norm: 0.9999999999999999
iter: 9491  x: [-2. -2.]  f(x): 5.656854249492381  grad at x: [-0.70710678 -0.70710678]  gradient norm: 0.9999999999999999
iter: 9492  x: [2. 2.]  f(x): 5.656854249492381  grad at x: [0.70710678 0.70710678]  gradient norm: 0.9999999999999999
iter: 9493  x: [-2. -2.]  f(x): 5.656854249492381  grad at x: [-0.70710678 -0.70710678]  gradient norm: 0.9999999999999999
iter: 9494  x: [2. 2.]  f(x): 5.656854249492381  grad at x: [0.70710678 0.70710678]  gradient norm: 

KeyboardInterrupt: ignored

In [ ]:
#Compute gradient descent with scaling and with backtracking line search 

print("Newton's method with Backtracking Line Search:")
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls, k, condition2 = find_minimizer_gdscaling_non_diagonal(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH_SCALING, 1.0, 0.5, 0.5)
iterations2 = range(k)
print("Optimizer: ", x_opt_bls)
print('Minimum Value: ', evalf(x_opt_bls))
print("Number of iterations: ", k)

Newton's method with Backtracking Line Search:
Optimizer:  [0. 0.]
Minimum Value:  4.0
Number of iterations:  1


### **Observations:**

***Using Newtons method with scaling and constant step length:*** \
Here we can observe that the x value is oscillating between (-2,-2) and (2,2) and so the gradient descent method does not seem to converge for this value of step length. \
Function value at (-2,-2) and (2,2) = 5.656854249492381 \
Gradient at (-2,-2) = [-0.70710678 ,-0.70710678] \
Gradient at (2,2) = [0.70710678 ,0.70710678] \
A possible reason for this can be the value of constant step length. It may be the case that this value is too large so the gradient descent is oscillating for one side to the other.

***Using Newton's method with Backtracking Line Search:*** \
Optimizer:  [0. 0.] \
Minimum Value:  4.0 \
Number of iterations:  1 \
Clearly we can see that in Newton's method with backtracking line search we got the minimizer in just one iteration. So we can conclude that Newton's method with back tracking is good approach than that of using constant step-length.

## **Answer 3:**

In [ ]:
#Compute gradient descent with scaling and with backtracking line search 

print("Gradient Descent method without scaling and step length by backtracking:")
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls, k, condition2 = find_minimizer_gd(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, 1.0, 0.5, 0.5)
iterations2 = range(k)
print("Optimizer: ", x_opt_bls)
print('Minimum Value: ', evalf(x_opt_bls))
print("Number of iterations: ", k)

Gradient Descent method without scaling and step length by backtracking:
Optimizer:  [7.62525638e-10 7.62525638e-10]
Minimum Value:  4.0
Number of iterations:  32


### **Observations:**
***Using Gradient Descent method without scaling and step length by backtracking:*** \
Optimizer:  [7.62525638e-10 7.62525638e-10] \
Minimum Value:  4.0 \
Number of iterations:  32 \

In the above part we observed that when we implemented Newton's method without scaling, it did not converge to the optimal solution while when scaling is done, it converges pretty fast that is in 1 step. \
From here we can see that Gradient Descent with backtracking line search method with scaling is better than Newton's method without scaling as it converged to the optimal solution in just 32 steps.\
From here we can infer: \

*Newton's Method without scaling < Gradient Descent without scaled backtracking < Newton's Method with scaling*

Also the Newton's method with scaling converges to the exact solution of the optimization problem, while with backtracking line search method we only reach within the tolerance level.



## **Answer 4:**

In [ ]:
start_x = np.array([8.0,8.0])
x,no_of_iterations,condition_no = find_minimizer_gdscaling_non_diagonal(start_x,my_tol,CONSTANT_STEP_LENGTH)

<ipython-input-2-7b8b0cfdf61a>:5: RuntimeWarning: overflow encountered in double_scalars
  return np.array([[4/(x[0]**2 + 4)**1.5,0] , [0,4/(x[1]**2 + 4)**1.5]])


ValueError: ignored

We are getting this error because here we are using  $\mathbf{D}^k = (\nabla^2f(\mathbf{x}))^{-1} $, so we need to find the inverse of the hessian matrix which is only possible if hessian is non-singular. But here our starting point is $[8.0,8.0]$ and while converging to optimizer there exist a value for which the hessian is singular.

In [ ]:
print("Newton's method with Backtracking Line Search:")
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls, k, condition2 = find_minimizer_gdscaling_non_diagonal(start_x, my_tol, BACKTRACKING_LINE_SEARCH_SCALING, alpha, rho, gamma)
iterations2 = range(k)
print("Optimizer: ", x_opt_bls)
print('Minimum Value: ', evalf(x_opt_bls))
print("Number of iterations: ", k)

Newton's method with Backtracking Line Search:
Optimizer:  [2.83764947e-12 2.83764947e-12]
Minimum Value:  4.0
Number of iterations:  13


### **Observations:**

***Using Newton's method with Constant Step Length:*** \
Here the method does not converge as the hessian becomes singular at some point along the way and so $D_k$ cannot be computed so the method fails. 


***Using Newton's method with Backtracking Line Search:*** \
Optimizer:  [2.83764947e-12 2.83764947e-12] \
Minimum Value:  4.0 \
Number of iterations:  13

In just 13 steps, Newton's method with Backtracking Line Search converges to the exact minimum value of the function i.e 4

## **Answer 5:**

In [ ]:
#Compute gradient descent with scaling and with backtracking line search 

print("Gradient Descent method without scaling and step length by backtracking:")
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls, k, condition2 = find_minimizer_gd(start_x, my_tol, BACKTRACKING_LINE_SEARCH, alpha,rho,gamma)
iterations2 = range(k)
print("Optimizer: ", x_opt_bls)
print('Minimum Value: ', evalf(x_opt_bls))
print("Number of iterations: ", k)

Gradient Descent method without scaling and step length by backtracking:
Optimizer:  [8.3177047e-10 8.3177047e-10]
Minimum Value:  4.0
Number of iterations:  39


### **Observations:**

***Using Gradient Descent method without scaling and step length by backtracking:*** \
Optimizer:  [8.3177047e-10 8.3177047e-10] \
Minimum Value:  4.0 \
Number of iterations: 39 \

Here also we observe a similar behaviour as above i.e. 

*Newton's Method without scaling < Gradient Descent without scaled backtracking < Newton's Method with scaling*

Gradient descent without scaling works better than Newton's method with constant step length and worse than Newton's method with scaling.